In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\guilh\AppData\Local\Temp\ipykernel_20568\2080034654.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
medicoes3 = pd.read_csv('./04-dados-qualar-longo-AML/medicoes-longo_seleccionadas_AML.csv', thousands=',', index_col=0, parse_dates=True)
medicoes3['Dia']= medicoes3.index.day #adicionar coluna com o Dia
medicoes3['Ano']=medicoes3.index.year #adicionar coluna com o Ano
medicoes3['Mes']=medicoes3.index.month #adicionar coluna com o Mes
medicoes3

,Station,Poluente,Valor,Dia,Ano,Mes
data,,,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1
...,...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022,12
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022,12
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022,12


In [3]:
#Criar o PM10 violacoes
PM10_violacoes = medicoes3[medicoes3['Poluente'] == 'PM10'] #filtrar o df
media_por_ano=PM10_violacoes.groupby(['Station', 'Ano'])['Valor'].mean().reset_index() #media por ano
media_por_dia=PM10_violacoes.groupby(['Station','Ano','Mes','Dia'])['Valor'].mean().reset_index() #media por dia
n_vio_ano=media_por_dia.groupby(['Station', 'Ano']).apply(lambda x: (x['Valor'] > 50).sum()).reset_index()#numero de violações por Ano de cada dia
PM10_violacoes=pd.merge(media_por_ano, n_vio_ano, on= ['Station','Ano'], how='left') #juntar os dfs
PM10_violacoes.rename(columns={'Valor':'Media_anual', 0:'Num_dias_excedidos'}, inplace=True)

PM10_violacoes

C:\Users\guilh\AppData\Local\Temp\ipykernel_20568\1051800089.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  n_vio_ano=media_por_dia.groupby(['Station', 'Ano']).apply(lambda x: (x['Valor'] > 50).sum()).reset_index()#numero de violações por Ano de cada dia


,Station,Ano,Media_anual,Num_dias_excedidos
0,Alto Seixalinho,2007,34.100929,32
1,Alto Seixalinho,2008,35.756789,45
2,Alto Seixalinho,2009,34.216716,42
3,Alto Seixalinho,2010,33.485939,26
4,Alto Seixalinho,2011,30.710909,17
...,...,...,...,...
310,Santa Cruz de Benfica,2018,28.376249,2
311,Santa Cruz de Benfica,2019,19.202870,6
312,Santa Cruz de Benfica,2020,17.916689,2
313,Santa Cruz de Benfica,2021,18.613222,8


In [4]:
#Função que diz o motivo

def determinar_motivo(row):
    if row['Media_anual'] > 40 and row['Num_dias_excedidos'] > 35:
        return 'ambos'
    elif row['Media_anual'] > 40:
        return 'media'
    elif row['Num_dias_excedidos'] > 35:
        return 'dias'
    else:
        return 'nenhum'

# Aplicar a função a cada linha do DataFrame
PM10_violacoes['Motivo'] = PM10_violacoes.apply(determinar_motivo, axis=1)


PM10_violacoes

,Station,Ano,Media_anual,Num_dias_excedidos,Motivo
0,Alto Seixalinho,2007,34.100929,32,nenhum
1,Alto Seixalinho,2008,35.756789,45,dias
2,Alto Seixalinho,2009,34.216716,42,dias
3,Alto Seixalinho,2010,33.485939,26,nenhum
4,Alto Seixalinho,2011,30.710909,17,nenhum
...,...,...,...,...,...
310,Santa Cruz de Benfica,2018,28.376249,2,nenhum
311,Santa Cruz de Benfica,2019,19.202870,6,nenhum
312,Santa Cruz de Benfica,2020,17.916689,2,nenhum
313,Santa Cruz de Benfica,2021,18.613222,8,nenhum


In [5]:
PM10_violacoes.to_csv('./04-dados-qualar-longo-AML/PM10_violacoes.csv', index=True)  #guardar o dataframe